# Direct Marketing with Keras

In [ ]:
# This package makes it easy to compute F1, precision and recall
!pip install keras-metrics

In [ ]:
import keras
import keras_metrics
import pandas as pd

## Load the dataset

In [ ]:
data = pd.read_csv('./bank-additional/bank-additional-full.csv', sep=';')
print(data.shape)

In [ ]:
data.head()

In [ ]:
one_class = data[data['y']=='yes']
one_class_count = one_class.shape[0]
print("Positive samples: %d" % one_class_count)

zero_class = data[data['y']=='no']
zero_class_count = zero_class.shape[0]
print("Negative samples: %d" % zero_class_count)

## Encode categorical variables

In [ ]:
data = pd.get_dummies(data)
data.head()

## Separate samples and labels

In [ ]:
X = data.drop(['y_yes', 'y_no'], axis=1)
Y = data['y_yes']

## Scale features

In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
X = pd.DataFrame(X)

In [ ]:
X.head()

In [ ]:
from sklearn import model_selection
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(X, Y, test_size=0.1, random_state=123)

## Build the model and train

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense

# Number of features in the training set
input_dim = x_train.shape[1]
print(input_dim)

model = Sequential()
model.add(Dense(16, input_dim=input_dim, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # because we want a probability between 0 and 1
# https://en.wikipedia.org/wiki/Sigmoid_function

model.compile(loss='binary_crossentropy', optimizer="adam", 
              metrics=['binary_accuracy', 
                    keras_metrics.precision(), 
                    keras_metrics.recall(),
                    keras_metrics.f1_score()])

print(model.summary())

model.fit(x_train, y_train, validation_data=(x_validation, y_validation), 
          epochs=20, batch_size=32)

## Score the model on the test set

In [ ]:
score = model.evaluate(x_validation, y_validation, batch_size=128)
for name,value in zip(model.metrics_names,score):
    print("%s %.2f" % (name,value))